In [1]:
import sys

sys.path.append('../../..')

1) Prepare the dataset in the pytorch Dataset format.
For example like this:

In [2]:
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.datasets import VOCSegmentation

transform = Compose([ToTensor(), Resize((500, 400))])
train_dataset = VOCSegmentation(
    root='/media/n31v/data/datasets',
    transform=transform,
    target_transform=transform,
)
val_dataset = VOCSegmentation(
    root='/media/n31v/data/datasets',
    image_set='val',
    transform=transform,
    target_transform=transform
)
image, mask = train_dataset.__getitem__(30)
print(image.shape, mask.shape)

torch.Size([3, 500, 400]) torch.Size([1, 500, 400])


2) Model initialization.

In [3]:
from torchvision.models.segmentation import deeplabv3_resnet50
from core.architecture.experiment.nn_experimenter import SegmentationExperimenter

experimenter = SegmentationExperimenter(
    model=deeplabv3_resnet50(num_classes=21), # Number of classes in your dataset
)

3) Fit parameters.

In [4]:
from torch.utils.data import DataLoader
from core.architecture.experiment.nn_experimenter import FitParameters

dl_params={'batch_size': 4, 'num_workers': 4}

fit_params = FitParameters(
    dataset_name='VOC',
    train_dl=DataLoader(train_dataset, shuffle=True, **dl_params),
    val_dl=DataLoader(val_dataset, **dl_params),
    num_epochs=50
)

4) Initialization of the structure optimization (optionally).

In [5]:
from core.operation.optimization.structure_optimization import SVDOptimization

svd_optimization = SVDOptimization(energy_thresholds=[0.1, 0.3, 0.5, 0.7, 0.9])

5) Running Model Training.
If you want to use structure optimization use ``svd_optimization.fit()`` instead of ``experimenter.fit()``

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import warnings
warnings.filterwarnings("ignore")

experimenter.fit(p=fit_params)
# svd_optimization.fit(exp=experimenter, params=fit_params)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import warnings
warnings.filterwarnings("ignore")

svd_optimization.fit(exp=experimenter, params=fit_params)

Now you can get predictions for all images in a folder. The method returns a dictionary {'image_name': class}.

In [ ]:
from core.architecture.datasets.prediction_datasets import PredictionFolderDataset

prediction_dataset = PredictionFolderDataset(
    image_folder='', # Path to your folder with images
    transform=transform,
)
prediction_dl = DataLoader(prediction_dataset, **dl_params)
preds = experimenter.predict(prediction_dl)
preds